# Main class for running the molecule environment

In [1]:
import gym
from gym import wrappers, logger
from MoleculeAgent import MoleculeAgent

In [2]:
def main():
    
    # Get input goal and start state from user
    goal = input("Enter a molecule to act as an optimisation Goal. \n for a random molecule enter '1' ")
    if goal == "1":
        mol = 1
    else: 
        mol = input("Enter an atom or molecule to act as the start state of the environment: ")
        
    env = gym.make("gym_molecule:molecule-v0")
    env.SetState(mol,goal)
    print("Environment initialiased.")
    #Create a new Molecule Agent object. 
    agent = MoleculeAgent(env.action_space)
    
    
    episode_count = 3
    reward = 0
    done = False

    #The 'RL algorithm' learns for 2 episodes of undefined length.
    for i in range(0, episode_count):
        print("EPISODE:", i)
        
        #The env resets.
        ob = env.reset()
        print("Env original state: ", ob)
        
        #While what is True? Maybe While done is False?
        for x in range(20):
            
            #The agent chooses its action based on the tuple from the current env state.
            action = agent.act(ob, reward, done)
            print()
            print("Agent's chosen action: ", action) 
            
            #The new tuple is returned because the env has considered the agent's action.
            ob, reward, done, _ = env.step(action)
            print("Env new state:", ob, "Reward to agent:",reward, "Done:",done, "Info: ", _)
            
            #The env renders. 
            env.render()
            
            
        
            #If the env reaches a terminating state, the current iteration ends and the next 
            #episode begins. 
            """"if done:
                print()
                break"""
        
    env.close()
    
    
if __name__ == '__main__':
    main()

Enter a molecule to act as an optimisation Goal. 
 for a random molecule enter '1' we
Enter an atom or molecule to act as the start state of the environment: w


AttributeError: 'MoleculeEnvironment' object has no attribute 'CheckGoal'